## 1. 用二元数实现自动微分

### 二元数是什么？

类似复数使用虚数单元 $i$ 来扩展实数，我们可以用一个性质略有不同的 $\epsilon$ 来扩展实数域。使得它恰好具有以下的性质

$$
\epsilon^2 = 0
$$

这样的性质会产生什么有趣的作用呢？我们不妨现在Julia里实现这样一种数。

首先定义它的类型，注意二元数应该是 `Number` 的一种，所以我们作为它的子类实现

In [1]:
struct Dual{T} <: Number
    value::T
    epsilon::T
end

这样我们就有一个类型用来存储一对数字 $(value, epsilon) = value + epsilon * \epsilon$

所以很自然的，我们有

In [2]:
const ϵ = Dual(false, true)

Dual{Bool}(false, true)

为了方便获取内部成员，我们定义一些property方法

In [3]:
value(x::Dual) = x.value
epsilon(x::Dual) = x.epsilon

epsilon (generic function with 1 method)

想想为什么使用property比点更general？

然后我们来定义它的运算规则：

乘法：

$$
(a, b) * (c, d) = (a + b * \epsilon) * (c + d * \epsilon) = ac + (ad + bc)\epsilon + bd\epsilon^2 = (ac, ad + bc)
$$

加法：

$$
(a, b) + (c, d) = (a + c, b + d)
$$

In [4]:
Base.:*(z::Dual, w::Dual) = Dual(value(z)*value(w), epsilon(z)*value(w)+value(z)*epsilon(w))
Base.:+(z::Dual, w::Dual) = Dual(value(z)+value(w), epsilon(z)+epsilon(w))

除法：

$$
\begin{aligned}
\frac{a + b\epsilon}{c + d\epsilon} &= \frac{(a + b\epsilon)(c - d\epsilon)}{(c + d\epsilon)(c - d\epsilon)} \\
                                    &= \frac{ac + (bc - ad)\epsilon}{c^2}\\
                                    &= \frac{a}{c} + \frac{bc - ad}{c^2}\epsilon
\end{aligned}
$$

In [5]:
Base.:/(z::Dual, w::Dual) = Dual(value(z)/value(w), (epsilon(z)*value(w)-value(z)*epsilon(w))/(value(w)*value(w)))

让我们来随便试试

In [6]:
Dual(1, 2) * ϵ

Dual{Int64}(0, 1)

这个打印出来的不是特别好看，让我们稍微修改一下打印出来的样式

In [7]:
Base.show(io::IO, x::Dual) = print(io, value(x), "+", epsilon(x), "ϵ")

现在我们再来试试

In [8]:
x = Dual(1.0, 1.0)
x^2

1.0+2.0ϵ

In [9]:
x^3

1.0+3.0ϵ

有没有发现什么规律？

没错，$\epsilon$ 的系数就是这个函数的导数，似乎我们只要向某个函数 $f$ 输入一个以 1 为系数的二元数 $x + \epsilon$ 就可以得到 $f'(x)$，为什么呢？

实际上这来自于摄动理论，$\epsilon$ 在这里的定义实际上是一阶摄动，它满足

$$
\lim \epsilon \rightarrow 0
$$

是一个无穷小量，而我们要求 $\epsilon^2$ 等于0就是为了忽略更高阶的摄动，所以我们将它带入Taylor级数就有

$$
f(a + b\epsilon) = f(a) + \frac{f'(a)}{1!}b \epsilon + \frac{f''(a)}{2!}(b\epsilon)^2 + \cdots
$$

如果我们要求 $\epsilon^2 = 0$ 我们就有

$$
f(a + b\epsilon) = f(a) + f'(a) b \epsilon
$$

很自然的，我们希望它能够在更复杂的函数上工作，比如这个

$$
f(x) = \frac{1 + x^2}{1 - x^2}
$$

In [10]:
f(x) = (1+x^2) / (1-x^2)

f (generic function with 1 method)

In [11]:
f(Dual(2, 1))

ErrorException: promotion of types Int64 and Dual{Int64} failed to change any arguments

报错了？？？

我们看一下错误信息，这里列出了程序的函数调用栈，在f函数中，调用了一个这样的method：`+(::Int64, ::Dual{Int64})`。这是因为f中计算了一个整数 1 加上 `x^2`， 我们不妨看看这个method

In [12]:
@edit +(1, x^2)

```julia
+(x::Number, y::Number) = +(promote(x,y)...)
*(x::Number, y::Number) = *(promote(x,y)...)
-(x::Number, y::Number) = -(promote(x,y)...)
/(x::Number, y::Number) = /(promote(x,y)...)
```

`promote` 是什么呢？让我们来看一下文档

In [13]:
?promote

search: promote promote_type promote_rule promote_shape pointer_from_objref



```
promote(xs...)
```

Convert all arguments to a common type, and return them all (as a tuple). If no arguments can be converted, an error is raised.

# Examples

```jldoctest
julia> promote(Int8(1), Float16(4.5), Float32(4.1))
(1.0f0, 4.5f0, 4.1f0)
```


所以我们有两种思路，一种思路是，当一个数 `x` 遇到对应的Dual的时候，我们自动把它提升为 `x + 0ϵ` 的二元数，或者我们直接定义Dual和其它Number的运算规则。这里似乎定义类型提升更加简单，这样我们不需要处理众多的运算规则。

In [14]:
Base.promote_rule(::Type{Dual{T}}, ::Type{T}) where {T <: Number} = Dual{T}

In [15]:
promote(Dual(1, 1), 1)

MethodError: MethodError: no method matching Dual{Int64}(::Int64)
Closest candidates are:
  Dual{Int64}(::T<:Number) where T<:Number at boot.jl:741
  Dual{Int64}(::Any, !Matched::Any) where T at In[1]:2
  Dual{Int64}(!Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:243
  ...

这个显然是缺少一个constructor，或者说缺少类型转换的定义，Julia不会进行自动类型转换，所以我们要显式定义它

In [16]:
Dual{T}(x) where T = Dual{T}(x, zero(T))

In [17]:
promote(Dual(2, 1), 1)

(2+1ϵ, 1+0ϵ)

再次尝试，我们没有为 `Dual` 定义减法运算符

In [18]:
f(Dual(2, 1))

ErrorException: - not defined for Dual{Int64}

In [19]:
Base.:-(z::Dual, w::Dual) = Dual(value(z)-value(w), epsilon(z)-epsilon(w))
Base.:-(z::Dual) = Dual(-value(z), -epsilon(z))

In [20]:
f(Dual(2, 1))

-1.6666666666666667+0.8888888888888888ϵ

成功！

## 2. 前馈神经网络（feed forward neural network）

在这个demo project里，我们将通过以下两个例子感受使用Julia编程：

- 使用Flux/Zygote的自动微分和优化器来训练一个简单的神经网络
- 从零搭建一个自己的神经网络

首先让我们来安装Julia的机器学习框架Flux


```julia
(1.1) pkg> add Flux
```

In [21]:
using Flux, Flux.Data.MNIST

imgs = MNIST.images();

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1186


我们可以看看这些图片都是什么

In [22]:
imgs[1]

然后我们将图片转换成一个向量，然后再组装成大的batch

In [23]:
X = hcat(float.(reshape.(imgs, :))...)

784×60000 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [24]:
labels = MNIST.labels()

60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 3
 5
 3
 ⋮
 7
 8
 9
 2
 9
 5
 1
 8
 3
 5
 6
 8

接下来我们需要把label编程onehot向量，方便和我们一会儿神经网络的输出做比较

In [25]:
using Flux: onehotbatch, onecold, crossentropy, throttle

In [26]:
Y = onehotbatch(labels, 0:9)

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

我们这里用一个非常简单的两层神经网络来作为我们的模型

In [27]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

然后用 交叉熵（cross entropy） 来定义损失函数（loss function）

In [28]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

我们直接用整个训练数据来训练，然后使用ADAM优化器。

In [29]:
dataset = Iterators.repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

然后我们看下 `train!` 函数是怎么用的

In [30]:
?Flux.train!

```
train!(loss, params, data, opt; cb)
```

For each datapoint `d` in `data` computes the gradient of `loss(d...)` through backpropagation and calls the optimizer `opt`.

Takes a callback as keyword argument `cb`. For example, this will print "training" every 10 seconds:

```julia
Flux.train!(loss, params, data, opt,
            cb = throttle(() -> println("training"), 10))
```

The callback can call `Flux.stop()` to interrupt the training loop.

Multiple optimisers and callbacks can be passed to `opt` and `cb` as arrays.


In [31]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

loss(X, Y) = 2.286031f0 (tracked)
loss(X, Y) = 1.0044526f0 (tracked)
loss(X, Y) = 0.5802763f0 (tracked)
loss(X, Y) = 0.44627106f0 (tracked)
loss(X, Y) = 0.38035807f0 (tracked)
loss(X, Y) = 0.33995047f0 (tracked)
loss(X, Y) = 0.3120375f0 (tracked)
loss(X, Y) = 0.29124454f0 (tracked)


现在我们定义我们模型最后训练出来的精度，也就是看看模型预测结果的准确性

In [32]:
using Statistics
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

accuracy (generic function with 1 method)

在训练集上，我们的精度是

In [33]:
accuracy(X, Y)

0.9230333333333334

在测试集上，我们的精度是

In [34]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...)
tY = onehotbatch(MNIST.labels(:test), 0:9)
accuracy(tX, tY)

0.9236

### 3. 以上的程序是如何工作的？

### 计算图

#### 前馈 （forward）

<img src="http://blog.rogerluo.me/images/comput-graph-forward.gif" height=50% width=60%></img>

#### 后馈 （backward）

<img src="http://blog.rogerluo.me/images/comput-graph-backward.gif" height=50% width=60%></img>

如何在代码里定义计算图呢？实际上很简单，我们只要定义图的节点（node）和变量（variable）就可以了

我们在每个节点里将其前馈输入的变量和函数都存下来，首先我们定义一个比较一般的抽象节点，因为每个节点都有一个值，我们将这个值的类型记为 `T`

In [35]:
abstract type AbstractNode{T} end

然后我们再来定义后向传播的规则，矩阵乘法 $A * B$ 的梯度分别是 $\Delta B^T$ 和 $A^T \Delta$，这也就意味着我们在回传梯度的时候需要前馈传播的 $A$ 和 $B$，那么要怎样缓存（cache）这两个变量以供后向传播的时候用呢？

建立一个 `Node` 里面存储被调用的函数 （乘法算符）和函数的变量

In [36]:
struct Node{T, F, Args} <: AbstractNode{T}
    output::T
    f::F
    args::Args
end

然后我们还需要定义叶子节点以标记哪些是变量（Variable）也就是需要计算梯度的节点

In [37]:
struct Variable{T} <: AbstractNode{T}
    data::T
    grad::T
end

为了方便起见，我们定义一个构造函数，让梯度默认为零

In [38]:
Variable(A) = Variable(A, zero(A))

Variable

我们希望任何表达式都会被记录进一个node中去，例如 `A * B`，那么当我定义了两个变量

In [39]:
A = Variable(rand(2, 3))
B = Variable(rand(3, 4));

的时候，如何让 `A * B` 构建出一个Node呢？

很简单，我们重载 `*` 这个运算符（其实就是重载函数）

In [40]:
Base.:(*)(x::AbstractNode, y::AbstractNode) = Node(*, x, y)
Base.:(*)(x::AbstractNode, y) = Node(*, x, y)
Base.:(+)(x::AbstractNode, y::AbstractNode) = Node(+, x, y)

为了方便重载运算符，我们同时定义一个 `Node` 的构造函数，和一个用于取出Variable和Node的值的函数 `data`

In [41]:
Node(f::Function, args...) = Node(f(map(data, args)...), f, args)
data(x::Variable) = x.data
data(x::Node) = x.output
data(x) = x

data (generic function with 3 methods)

让我们来试一试

In [42]:
A * B

Node{Array{Float64,2},typeof(*),Tuple{Variable{Array{Float64,2}},Variable{Array{Float64,2}}}}([0.748665 0.52093 0.431819 0.788467; 0.394618 0.30398 0.302953 0.344516], *, (Variable{Array{Float64,2}}([0.945043 0.265197 0.640297; 0.565866 0.364138 0.165449], [0.0 0.0 0.0; 0.0 0.0 0.0]), Variable{Array{Float64,2}}([0.587719 0.467632 0.0495658 0.233593; 0.0409801 0.0641033 0.593445 0.222048; 0.284832 0.0968261 0.355456 0.794671], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0])))

为了后面阅读方便，我们不妨重新定义一下Node和Variable的打印格式

In [43]:
function Base.show(io::IO, x::AbstractNode{<:AbstractArray})
    summary(io, x)
    println(io)
    Base.print_array(io, data(x))
end

In [44]:
A * B

Node{Array{Float64,2},typeof(*),Tuple{Variable{Array{Float64,2}},Variable{Array{Float64,2}}}}
 0.7486649597745116  0.5209301520756134  …  0.788467055435203
 0.3946180633192462  0.3039796056538721     0.344515816197736

有了计算图以后我们就可以开始着手考虑如何进行后向传播了，我们的目标是实现如下的功能：

给定梯度输入，求所有Variable的梯度，并且将梯度累加到Variable中的 `grad` 变量上去

假如开始后向传播的节点就是 `Variable` 这非常简单

In [45]:
function backward(x::Variable, Δ)
    x.grad .+= Δ
    return
end

backward (generic function with 1 method)

对于不是计算图中的节点，也非常简单

In [46]:
backward(x, Δ) = nothing

backward (generic function with 2 methods)

而如果我们是从更加一般的 `Node` 开始反馈，那么就需要递归的对每个输入进行遍历，我们假设每个输入的梯度可以用 `gradient` 函数获得

In [47]:
function backward(x::Node, Δ)
    grads = gradient(x.f, Δ, x.output, map(data, x.args)...)
    # number of gradients is the same with number of inputs
    for (each, grad) in zip(x.args, grads)
        backward(each, grad)
    end
    return
end

backward (generic function with 3 methods)

剩下的只需要为各个不同的函数定义梯度了，我们这里为一个简单前馈神经网络所需的函数定义梯度

In [82]:
function gradient(::typeof(*), Δ, output, A, B)
    return reshape(Δ * B', size(A)), reshape(A' * Δ, size(B))
end

gradient(::typeof(+), Δ, output, A, B) = (Δ, Δ)

sigmoid(x::AbstractVecOrMat) = @. one(x) / (one(x) + exp(-x))
sigmoid(x::AbstractNode{<:AbstractArray}) = Node(sigmoid, x)
gradient(::typeof(sigmoid), Δ, output, x) = (@.(Δ * (one(x) - output) * output), )

gradient (generic function with 3 methods)

接下来就是见证奇迹的时刻！

让我们来计算一个 Dense 层：

$$
Dense(x) = \sigma (Wx + b)
$$

In [83]:
W = Variable(rand(4, 4))
b = Variable(rand(4))

Dense(x) = sigmoid(W * x + b)

x = rand(4)
z = Dense(x)

Node{Array{Float64,1},typeof(sigmoid),Tuple{Node{Array{Float64,1},typeof(+),Tuple{Node{Array{Float64,1},typeof(*),Tuple{Variable{Array{Float64,2}},Array{Float64,1}}},Variable{Array{Float64,1}}}}}}
 0.7207142161532042
 0.7467479910460055
 0.6610597120987938
 0.775920740230816 

然后对某个输入的梯度进行后向传播

In [84]:
Δ = rand(4)
backward(z, Δ)
W.grad

4×4 Array{Float64,2}:
 0.000454981  0.000360364  0.000487597  0.000475926
 0.0302546    0.023963     0.0324235    0.0316474  
 0.00666228   0.0052768    0.00713987   0.00696898 
 0.038584     0.0305601    0.0413499    0.0403602  

最后让我们和Flux的自动微分比较一下结果

In [88]:
using Flux.Tracker, LinearAlgebra

Flux_W = param(W.data)
Flux_b = param(b.data)
Flux_Dense(x) = sigmoid(Flux_W * x + Flux_b)
z = Flux_Dense(x)
back!(z, Δ)

norm(Flux_W.grad - W.grad)

8.76210447256141e-18